# Chapter 2 - Exercise 3: Bioreactor Scale-up Analysis

**VUB Master Bioengineering - Biofabrication Course**

---

## Learning Objectives
- Understand scaling principles from laboratory (mL) to production (L) volumes
- Analyze oxygen transfer rates and cell density limitations
- Compare different bioreactor configurations and their economic implications
- Practice unit conversions and scaling calculations
- Optimize cell density vs production volume trade-offs
- Apply bioprocess engineering principles to scaffold-free biofabrication

## Background Theory

From **Chapter 2.1.2**, we learned that **bioreactors** provide controlled environments for scaffold-free tissue growth. Key challenges in scaling up include:

- **Mass Transport**: Ensuring adequate oxygen and nutrient delivery
- **Mechanical Environment**: Maintaining appropriate shear stress and mixing
- **Cell Density**: Balancing productivity with cell viability
- **Economic Factors**: Cost of equipment, labor, and consumables

**Figure 2.3** shows examples of stirred bioreactors and hollow fiber systems used for cell expansion. This exercise explores the quantitative relationships governing bioreactor design and scale-up.

---

## Instructions for Students

1. **Save a copy**: File → Save a copy in Drive
2. **Run cells in order**: Use Shift+Enter or click the play button
3. **Modify parameters**: Look for comments with `# MODIFY THIS`
4. **Answer questions**: Complete the analysis section at the end
5. **Experiment**: Try different scale-up scenarios and observe changes

# Section 1: Setup and Imports

Run this cell first to install required packages and import libraries.

In [ ]:
# Install required packages (run this cell first!)
!pip install matplotlib seaborn pandas numpy scipy plotly --quiet

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize_scalar, minimize
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11

print("Setup complete! Ready to analyze bioreactor scale-up.")
print("Packages loaded: numpy, matplotlib, pandas, seaborn, scipy, plotly")

# Section 2: Scaling Fundamentals

Core relationships for bioreactor scale-up based on mass transfer and mixing principles.

In [ ]:
# Fundamental scaling relationships

def oxygen_transfer_coefficient(volume_L, agitation_rpm, aeration_rate_vvm):
    """
    Calculate volumetric oxygen transfer coefficient (kLa) based on empirical correlations.
    
    Parameters:
    volume_L: bioreactor volume in liters
    agitation_rpm: stirrer speed in RPM
    aeration_rate_vvm: air flow rate in volumes per volume per minute
    
    Returns:
    kLa: oxygen transfer coefficient (1/hour)
    """
    # Empirical correlation: kLa = k * (P/V)^a * (Vs)^b
    # Simplified for educational purposes
    
    # Power per unit volume (simplified)
    power_per_volume = (agitation_rpm / 100) ** 1.5  # Normalized power input
    
    # Superficial gas velocity effect
    gas_velocity_effect = aeration_rate_vvm ** 0.7
    
    # Scale effect (larger reactors have lower kLa)
    scale_factor = (volume_L / 1.0) ** (-0.2)
    
    kLa = 10 * power_per_volume * gas_velocity_effect * scale_factor
    
    return kLa

def oxygen_consumption_rate(cell_density_per_mL, cell_type='mesenchymal_stem_cell'):
    """
    Calculate oxygen consumption rate based on cell density and type.
    
    Parameters:
    cell_density_per_mL: number of cells per mL
    cell_type: type of cell (affects metabolic rate)
    
    Returns:
    OUR: oxygen uptake rate (mg O2/L/hour)
    """
    # Specific oxygen uptake rates (mg O2/10^6 cells/hour)
    cell_specific_rates = {
        'mesenchymal_stem_cell': 0.05,
        'chondrocyte': 0.08,
        'hepatocyte': 0.15,
        'cardiomyocyte': 0.20,
        'fibroblast': 0.03
    }
    
    specific_rate = cell_specific_rates.get(cell_type, 0.05)
    
    # Convert to mg O2/L/hour
    OUR = (cell_density_per_mL * 1000) * (specific_rate / 1e6)
    
    return OUR

def calculate_maximum_cell_density(kLa, dissolved_oxygen_mg_L=8.0, critical_oxygen_mg_L=2.0, cell_type='mesenchymal_stem_cell'):
    """
    Calculate maximum sustainable cell density based on oxygen transfer limitations.
    
    Parameters:
    kLa: oxygen transfer coefficient (1/hour)
    dissolved_oxygen_mg_L: dissolved oxygen concentration (mg/L)
    critical_oxygen_mg_L: minimum oxygen level for cell survival (mg/L)
    cell_type: type of cell
    
    Returns:
    max_density: maximum cell density (cells/mL)
    """
    # Oxygen transfer rate (OTR) = kLa * (DO_sat - DO_critical)
    available_oxygen = dissolved_oxygen_mg_L - critical_oxygen_mg_L
    OTR = kLa * available_oxygen  # mg O2/L/hour
    
    # Cell specific rates
    cell_specific_rates = {
        'mesenchymal_stem_cell': 0.05,
        'chondrocyte': 0.08,
        'hepatocyte': 0.15,
        'cardiomyocyte': 0.20,
        'fibroblast': 0.03
    }
    
    specific_rate = cell_specific_rates.get(cell_type, 0.05)  # mg O2/10^6 cells/hour
    
    # Calculate maximum density
    max_density = (OTR * 1e6) / (specific_rate * 1000)  # cells/mL
    
    return max_density

def bioreactor_cost_model(volume_L, reactor_type='stirred_tank'):
    """
    Estimate bioreactor costs based on volume and type.
    
    Parameters:
    volume_L: bioreactor volume in liters
    reactor_type: type of bioreactor
    
    Returns:
    costs: dictionary of cost components (EUR)
    """
    
    # Base costs (simplified model)
    if reactor_type == 'stirred_tank':
        base_cost_per_L = 500  # EUR per liter capacity
        operating_cost_per_L_day = 10  # EUR per liter per day
        
    elif reactor_type == 'hollow_fiber':
        base_cost_per_L = 800  # Higher initial cost
        operating_cost_per_L_day = 15  # Higher operating cost
        
    elif reactor_type == 'perfusion':
        base_cost_per_L = 1200  # Highest complexity
        operating_cost_per_L_day = 20
        
    else:  # wave/rocking
        base_cost_per_L = 300
        operating_cost_per_L_day = 8
    
    # Scale factor (economies of scale)
    scale_factor = (volume_L / 10) ** 0.7  # Economy of scale
    
    capital_cost = base_cost_per_L * volume_L * scale_factor
    daily_operating_cost = operating_cost_per_L_day * volume_L
    
    # Additional costs
    medium_cost_per_L = 50  # Culture medium cost
    labor_cost_per_day = 200  # Operator cost
    
    costs = {
        'capital_cost_EUR': capital_cost,
        'daily_operating_EUR': daily_operating_cost,
        'medium_cost_per_batch_EUR': medium_cost_per_L * volume_L,
        'labor_cost_per_day_EUR': labor_cost_per_day,
        'total_daily_cost_EUR': daily_operating_cost + labor_cost_per_day
    }
    
    return costs

print("Scaling functions defined:")
print("• oxygen_transfer_coefficient() - calculates kLa")
print("• oxygen_consumption_rate() - calculates OUR")
print("• calculate_maximum_cell_density() - oxygen-limited density")
print("• bioreactor_cost_model() - economic analysis")

# Section 3: Scale-up Scenarios (Students: Modify These!)

Compare different scale-up strategies from lab bench to production.

In [ ]:
# =============================================================================
# STUDENT SCALE-UP SCENARIOS - MODIFY THESE VALUES!
# =============================================================================

# Laboratory Scale (starting point)
lab_scale = {
    'volume_L': 0.5,              # MODIFY THIS (0.1-2 L)
    'agitation_rpm': 100,         # MODIFY THIS (50-200 RPM)
    'aeration_vvm': 0.1,          # MODIFY THIS (0.05-0.5 vvm)
    'target_cell_density': 1e6,   # MODIFY THIS (1e5-5e6 cells/mL)
    'cell_type': 'mesenchymal_stem_cell',  # MODIFY THIS
    'culture_time_days': 7         # MODIFY THIS (3-14 days)
}

# Pilot Scale
pilot_scale = {
    'volume_L': 50,               # MODIFY THIS (10-100 L)
    'agitation_rpm': 80,          # MODIFY THIS (40-150 RPM)
    'aeration_vvm': 0.2,          # MODIFY THIS (0.1-0.8 vvm)
    'target_cell_density': 1e6,   # MODIFY THIS
    'cell_type': 'mesenchymal_stem_cell',  # MODIFY THIS
    'culture_time_days': 10        # MODIFY THIS
}

# Production Scale
production_scale = {
    'volume_L': 1000,             # MODIFY THIS (500-5000 L)
    'agitation_rpm': 60,          # MODIFY THIS (30-100 RPM)
    'aeration_vvm': 0.3,          # MODIFY THIS (0.2-1.0 vvm)
    'target_cell_density': 8e5,   # MODIFY THIS
    'cell_type': 'mesenchymal_stem_cell',  # MODIFY THIS
    'culture_time_days': 14        # MODIFY THIS
}

# Available cell types (for modification)
available_cell_types = [
    'mesenchymal_stem_cell',
    'chondrocyte', 
    'hepatocyte',
    'cardiomyocyte',
    'fibroblast'
]

# =============================================================================

# Analyze each scale
scales = {'Laboratory': lab_scale, 'Pilot': pilot_scale, 'Production': production_scale}
results = {}

print("BIOREACTOR SCALE-UP ANALYSIS")
print("=" * 50)

for scale_name, params in scales.items():
    # Calculate performance metrics
    kLa = oxygen_transfer_coefficient(params['volume_L'], params['agitation_rpm'], params['aeration_vvm'])
    max_density = calculate_maximum_cell_density(kLa, cell_type=params['cell_type'])
    our = oxygen_consumption_rate(params['target_cell_density'], params['cell_type'])
    costs = bioreactor_cost_model(params['volume_L'], 'stirred_tank')
    
    # Total cell yield
    total_cells = params['target_cell_density'] * params['volume_L'] * 1000  # total cells
    
    # Oxygen limitation check
    oxygen_limited = params['target_cell_density'] > max_density
    
    # Store results
    results[scale_name] = {
        'volume_L': params['volume_L'],
        'kLa_per_hour': kLa,
        'max_density_cells_per_mL': max_density,
        'target_density_cells_per_mL': params['target_cell_density'],
        'oxygen_limited': oxygen_limited,
        'our_mg_O2_per_L_per_hour': our,
        'total_cells': total_cells,
        'cells_per_day': total_cells / params['culture_time_days'],
        'capital_cost_EUR': costs['capital_cost_EUR'],
        'daily_cost_EUR': costs['total_daily_cost_EUR'],
        'cost_per_million_cells_EUR': (costs['total_daily_cost_EUR'] * params['culture_time_days']) / (total_cells / 1e6)
    }
    
    print(f"\n{scale_name} Scale ({params['volume_L']} L):")
    print(f"  kLa: {kLa:.1f} /hour")
    print(f"  Max cell density: {max_density:.1e} cells/mL")
    print(f"  Target density: {params['target_cell_density']:.1e} cells/mL")
    print(f"  Oxygen limited: {'YES' if oxygen_limited else 'NO'}")
    print(f"  Total cells: {total_cells:.1e}")
    print(f"  Daily output: {total_cells / params['culture_time_days']:.1e} cells/day")
    print(f"  Cost per million cells: €{(costs['total_daily_cost_EUR'] * params['culture_time_days']) / (total_cells / 1e6):.2f}")

print("\nTIP: Modify the parameters above and re-run to explore different scenarios!")
print(f"Available cell types: {', '.join(available_cell_types)}")

# Section 4: Scale-up Visualization

Visualize the relationships between scale, performance, and economics.

In [ ]:
# Create comprehensive scale-up analysis plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Bioreactor Scale-up Analysis', fontsize=16, fontweight='bold')

# Prepare data for plotting
scale_names = list(results.keys())
volumes = [results[s]['volume_L'] for s in scale_names]
kla_values = [results[s]['kLa_per_hour'] for s in scale_names]
max_densities = [results[s]['max_density_cells_per_mL'] for s in scale_names]
target_densities = [results[s]['target_density_cells_per_mL'] for s in scale_names]
total_outputs = [results[s]['total_cells'] for s in scale_names]
costs_per_million = [results[s]['cost_per_million_cells_EUR'] for s in scale_names]

# Plot 1: kLa vs Volume (mass transfer limitation)
axes[0,0].semilogx(volumes, kla_values, 'bo-', linewidth=3, markersize=10)
for i, name in enumerate(scale_names):
    axes[0,0].annotate(name, (volumes[i], kla_values[i]), 
                      xytext=(10, 10), textcoords='offset points', fontweight='bold')

axes[0,0].set_xlabel('Bioreactor Volume (L)', fontweight='bold')
axes[0,0].set_ylabel('kLa (1/hour)', fontweight='bold')
axes[0,0].set_title('Oxygen Transfer vs Scale', fontweight='bold')
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Cell density comparison (oxygen limitation)
x_pos = np.arange(len(scale_names))
width = 0.35

bars1 = axes[0,1].bar(x_pos - width/2, np.array(max_densities)/1e6, width, 
                     label='Max Achievable', alpha=0.8, color='lightblue')
bars2 = axes[0,1].bar(x_pos + width/2, np.array(target_densities)/1e6, width,
                     label='Target', alpha=0.8, color='orange')

# Highlight oxygen-limited cases
for i, scale_name in enumerate(scale_names):
    if results[scale_name]['oxygen_limited']:
        axes[0,1].text(i, max(max_densities[i], target_densities[i])/1e6 + 0.1, 
                      'O₂ LIMITED', ha='center', color='red', fontweight='bold')

axes[0,1].set_xlabel('Scale', fontweight='bold')
axes[0,1].set_ylabel('Cell Density (10⁶ cells/mL)', fontweight='bold')
axes[0,1].set_title('Cell Density Limitations', fontweight='bold')
axes[0,1].set_xticks(x_pos)
axes[0,1].set_xticklabels(scale_names)
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Total output vs scale
axes[1,0].semilogy(volumes, np.array(total_outputs), 'go-', linewidth=3, markersize=10)
for i, name in enumerate(scale_names):
    axes[1,0].annotate(f'{total_outputs[i]:.1e}', (volumes[i], total_outputs[i]), 
                      xytext=(10, 10), textcoords='offset points', fontweight='bold')

axes[1,0].set_xlabel('Bioreactor Volume (L)', fontweight='bold')
axes[1,0].set_ylabel('Total Cell Output', fontweight='bold')
axes[1,0].set_title('Production Capacity', fontweight='bold')
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Cost per million cells
bars = axes[1,1].bar(scale_names, costs_per_million, alpha=0.8, 
                    color=['red', 'yellow', 'green'])

# Add value labels
for bar, cost in zip(bars, costs_per_million):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2, height + height*0.02,
                  f'€{cost:.2f}', ha='center', va='bottom', fontweight='bold')

axes[1,1].set_ylabel('Cost per Million Cells (EUR)', fontweight='bold')
axes[1,1].set_title('Production Economics', fontweight='bold')
axes[1,1].tick_params(axis='x', rotation=0)
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Scale-up Insights:")
print("• kLa generally decreases with scale (mass transfer challenge)")
print("• Oxygen limitation may require lower cell densities at large scale")
print("• Total output increases dramatically with scale")
print("• Cost per unit often decreases with scale (economies of scale)")

# Section 5: Optimization Analysis

Find optimal operating conditions for different objectives.

In [ ]:
# Optimization functions

def optimize_agitation_aeration(volume_L, cell_type, objective='productivity'):
    """
    Optimize agitation and aeration for given objective.
    
    Parameters:
    volume_L: reactor volume
    cell_type: type of cells
    objective: 'productivity', 'cost', or 'density'
    
    Returns:
    optimal conditions and performance
    """
    
    # Parameter ranges
    rpm_range = np.linspace(40, 150, 20)
    vvm_range = np.linspace(0.1, 0.8, 15)
    
    best_value = 0 if objective in ['productivity', 'density'] else float('inf')
    best_params = None
    results_grid = []
    
    for rpm in rpm_range:
        for vvm in vvm_range:
            # Calculate performance
            kLa = oxygen_transfer_coefficient(volume_L, rpm, vvm)
            max_density = calculate_maximum_cell_density(kLa, cell_type=cell_type)
            
            # Operating cost (simplified: higher RPM and VVM cost more)
            power_cost = (rpm / 100) ** 2 * volume_L * 0.1  # EUR/day
            air_cost = vvm * volume_L * 0.05  # EUR/day
            total_cost = power_cost + air_cost
            
            # Productivity (cells per day per EUR)
            if total_cost > 0:
                productivity = (max_density * volume_L * 1000) / total_cost
            else:
                productivity = 0
            
            results_grid.append({
                'rpm': rpm,
                'vvm': vvm,
                'kLa': kLa,
                'max_density': max_density,
                'cost': total_cost,
                'productivity': productivity
            })
            
            # Check if this is the best condition
            if objective == 'productivity' and productivity > best_value:
                best_value = productivity
                best_params = (rpm, vvm)
            elif objective == 'cost' and total_cost < best_value:
                best_value = total_cost
                best_params = (rpm, vvm)
            elif objective == 'density' and max_density > best_value:
                best_value = max_density
                best_params = (rpm, vvm)
    
    return best_params, best_value, results_grid

# Run optimization for production scale
objectives = ['productivity', 'cost', 'density']
optimization_results = {}

print("OPTIMIZATION ANALYSIS FOR PRODUCTION SCALE")
print("=" * 50)

volume = production_scale['volume_L']
cell_type = production_scale['cell_type']

for obj in objectives:
    best_params, best_value, grid = optimize_agitation_aeration(volume, cell_type, obj)
    optimization_results[obj] = {
        'best_params': best_params,
        'best_value': best_value,
        'grid': grid
    }
    
    print(f"\nOptimization for {obj.upper()}:")
    print(f"  Best RPM: {best_params[0]:.1f}")
    print(f"  Best VVM: {best_params[1]:.2f}")
    if obj == 'productivity':
        print(f"  Best productivity: {best_value:.1e} cells/day/EUR")
    elif obj == 'cost':
        print(f"  Minimum cost: €{best_value:.2f}/day")
    elif obj == 'density':
        print(f"  Maximum density: {best_value:.1e} cells/mL")

# Visualize optimization results
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
fig.suptitle('Optimization Landscapes for Production Scale Bioreactor', fontsize=14, fontweight='bold')

for i, obj in enumerate(objectives):
    grid_data = optimization_results[obj]['grid']
    df_grid = pd.DataFrame(grid_data)
    
    # Create pivot table for heatmap
    if obj == 'productivity':
        pivot = df_grid.pivot_table(values='productivity', index='vvm', columns='rpm')
        title = 'Productivity\n(cells/day/EUR)'
        cmap = 'viridis'
    elif obj == 'cost':
        pivot = df_grid.pivot_table(values='cost', index='vvm', columns='rpm')
        title = 'Operating Cost\n(EUR/day)'
        cmap = 'viridis_r'  # Reverse for cost (lower is better)
    else:  # density
        pivot = df_grid.pivot_table(values='max_density', index='vvm', columns='rpm')
        title = 'Max Cell Density\n(cells/mL)'
        cmap = 'plasma'
    
    # Create heatmap
    im = axes[i].imshow(pivot.values, cmap=cmap, aspect='auto', origin='lower')
    
    # Mark optimal point
    best_rpm, best_vvm = optimization_results[obj]['best_params']
    # Find closest indices
    rpm_idx = np.argmin(np.abs(pivot.columns - best_rpm))
    vvm_idx = np.argmin(np.abs(pivot.index - best_vvm))
    axes[i].plot(rpm_idx, vvm_idx, 'r*', markersize=20, markeredgecolor='white', markeredgewidth=2)
    
    # Set labels
    axes[i].set_title(title, fontweight='bold')
    axes[i].set_xlabel('Agitation (RPM)', fontweight='bold')
    if i == 0:
        axes[i].set_ylabel('Aeration (VVM)', fontweight='bold')
    
    # Set tick labels
    n_ticks = 5
    rpm_ticks = np.linspace(0, len(pivot.columns)-1, n_ticks, dtype=int)
    vvm_ticks = np.linspace(0, len(pivot.index)-1, n_ticks, dtype=int)
    axes[i].set_xticks(rpm_ticks)
    axes[i].set_yticks(vvm_ticks)
    axes[i].set_xticklabels([f'{pivot.columns[idx]:.0f}' for idx in rpm_ticks])
    axes[i].set_yticklabels([f'{pivot.index[idx]:.2f}' for idx in vvm_ticks])
    
    # Add colorbar
    plt.colorbar(im, ax=axes[i], shrink=0.8)

plt.tight_layout()
plt.show()

print("\nOptimization Insights:")
print("• Different objectives require different operating conditions")
print("• Trade-offs exist between productivity, cost, and cell density")
print("• Red stars (*) mark optimal conditions for each objective")
print("• Higher agitation/aeration improves mass transfer but increases costs")

# Section 6: Bioreactor Type Comparison

Compare different bioreactor configurations for scale-up.

In [ ]:
# Compare different bioreactor types
reactor_types = ['stirred_tank', 'hollow_fiber', 'perfusion', 'wave']
comparison_volume = 100  # L

reactor_comparison = {}

# Reactor-specific parameters
reactor_specs = {
    'stirred_tank': {
        'typical_kLa': 15,
        'cell_density_factor': 1.0,
        'mixing_quality': 0.8,
        'scalability': 0.9,
        'complexity': 0.5
    },
    'hollow_fiber': {
        'typical_kLa': 25,
        'cell_density_factor': 3.0,  # Higher density possible
        'mixing_quality': 0.6,
        'scalability': 0.6,
        'complexity': 0.8
    },
    'perfusion': {
        'typical_kLa': 20,
        'cell_density_factor': 2.0,
        'mixing_quality': 0.9,
        'scalability': 0.7,
        'complexity': 0.9
    },
    'wave': {
        'typical_kLa': 8,
        'cell_density_factor': 0.8,
        'mixing_quality': 0.7,
        'scalability': 0.4,  # Limited scalability
        'complexity': 0.3
    }
}

print("BIOREACTOR TYPE COMPARISON")
print(f"Comparison volume: {comparison_volume} L")
print("=" * 50)

for reactor_type in reactor_types:
    specs = reactor_specs[reactor_type]
    costs = bioreactor_cost_model(comparison_volume, reactor_type)
    
    # Calculate effective cell density
    base_density = 1e6  # cells/mL
    effective_density = base_density * specs['cell_density_factor']
    
    # Total output
    total_output = effective_density * comparison_volume * 1000
    
    # Performance score (weighted combination)
    performance_score = (
        0.3 * specs['typical_kLa'] / 25 +  # Normalize to hollow fiber
        0.2 * specs['cell_density_factor'] +
        0.2 * specs['mixing_quality'] +
        0.2 * specs['scalability'] +
        0.1 * (1 - specs['complexity'])  # Lower complexity is better
    )
    
    reactor_comparison[reactor_type] = {
        'kLa': specs['typical_kLa'],
        'effective_density': effective_density,
        'total_output': total_output,
        'capital_cost': costs['capital_cost_EUR'],
        'daily_cost': costs['total_daily_cost_EUR'],
        'cost_per_million_cells': (costs['total_daily_cost_EUR'] * 7) / (total_output / 1e6),
        'performance_score': performance_score,
        'mixing_quality': specs['mixing_quality'],
        'scalability': specs['scalability'],
        'complexity': specs['complexity']
    }
    
    print(f"\n{reactor_type.replace('_', ' ').title()}:")
    print(f"  kLa: {specs['typical_kLa']} /hour")
    print(f"  Effective density: {effective_density:.1e} cells/mL")
    print(f"  Total output: {total_output:.1e} cells")
    print(f"  Capital cost: €{costs['capital_cost_EUR']:,.0f}")
    print(f"  Daily cost: €{costs['total_daily_cost_EUR']:.0f}")
    print(f"  Cost per million cells: €{(costs['total_daily_cost_EUR'] * 7) / (total_output / 1e6):.2f}")
    print(f"  Performance score: {performance_score:.2f}")

# Create comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Bioreactor Type Comparison', fontsize=16, fontweight='bold')

reactor_names = [r.replace('_', ' ').title() for r in reactor_types]
colors = ['blue', 'red', 'green', 'orange']

# Plot 1: Cost comparison
capital_costs = [reactor_comparison[r]['capital_cost']/1000 for r in reactor_types]  # in k€
daily_costs = [reactor_comparison[r]['daily_cost'] for r in reactor_types]

x = np.arange(len(reactor_types))
width = 0.35

bars1 = axes[0,0].bar(x - width/2, capital_costs, width, label='Capital (k€)', alpha=0.8)
bars2 = axes[0,0].bar(x + width/2, daily_costs, width, label='Daily (€)', alpha=0.8)

axes[0,0].set_xlabel('Reactor Type', fontweight='bold')
axes[0,0].set_ylabel('Cost', fontweight='bold')
axes[0,0].set_title('Cost Comparison', fontweight='bold')
axes[0,0].set_xticks(x)
axes[0,0].set_xticklabels(reactor_names, rotation=45, ha='right')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Performance radar chart
metrics = ['kLa', 'mixing_quality', 'scalability', 'complexity']
angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

# Normalize metrics for radar chart
for i, reactor_type in enumerate(reactor_types[:2]):  # Show first 2 for clarity
    values = [
        reactor_comparison[reactor_type]['kLa'] / 25,  # Normalize
        reactor_comparison[reactor_type]['mixing_quality'],
        reactor_comparison[reactor_type]['scalability'],
        1 - reactor_comparison[reactor_type]['complexity']  # Invert complexity
    ]
    values += values[:1]  # Complete the circle
    
    axes[0,1].plot(angles, values, 'o-', linewidth=2, label=reactor_names[i], color=colors[i])
    axes[0,1].fill(angles, values, alpha=0.25, color=colors[i])

axes[0,1].set_xticks(angles[:-1])
axes[0,1].set_xticklabels(['kLa', 'Mixing', 'Scalability', 'Simplicity'])
axes[0,1].set_ylim(0, 1)
axes[0,1].set_title('Performance Radar (Stirred Tank vs Hollow Fiber)', fontweight='bold')
axes[0,1].legend()
axes[0,1].grid(True)

# Plot 3: Output vs Cost efficiency
outputs = [reactor_comparison[r]['total_output']/1e9 for r in reactor_types]  # in billions
cost_efficiency = [reactor_comparison[r]['cost_per_million_cells'] for r in reactor_types]

scatter = axes[1,0].scatter(cost_efficiency, outputs, s=150, c=colors, alpha=0.7)
for i, name in enumerate(reactor_names):
    axes[1,0].annotate(name, (cost_efficiency[i], outputs[i]),
                      xytext=(10, 10), textcoords='offset points', fontweight='bold')

axes[1,0].set_xlabel('Cost per Million Cells (€)', fontweight='bold')
axes[1,0].set_ylabel('Total Output (Billion Cells)', fontweight='bold')
axes[1,0].set_title('Cost Efficiency vs Output', fontweight='bold')
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Overall performance score
performance_scores = [reactor_comparison[r]['performance_score'] for r in reactor_types]
bars = axes[1,1].bar(reactor_names, performance_scores, color=colors, alpha=0.8)

# Add value labels
for bar, score in zip(bars, performance_scores):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2, height + 0.01,
                  f'{score:.2f}', ha='center', va='bottom', fontweight='bold')

axes[1,1].set_ylabel('Performance Score', fontweight='bold')
axes[1,1].set_title('Overall Performance Ranking', fontweight='bold')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)
axes[1,1].set_ylim(0, max(performance_scores) * 1.2)

plt.tight_layout()
plt.show()

# Recommend best reactor for different scenarios
print("\nRECOMMENDATIONS:")
print("\n• For MAXIMUM CELL DENSITY: Hollow Fiber")
print("• For LOWEST COST: Wave Bioreactor (small scale)")
print("• For LARGE SCALE PRODUCTION: Stirred Tank")
print("• For HIGH-VALUE PRODUCTS: Perfusion")
print("• For R&D/FLEXIBILITY: Stirred Tank")

# Section 7: Analysis Questions for Students

Complete these questions based on your scale-up analysis results.

## **Question 1: Oxygen Transfer Limitation**
Why does kLa generally decrease as bioreactor volume increases? How does this affect maximum achievable cell density at large scale?

*Write your answer here:*


---

## **Question 2: Scale-up Strategy**
You need to scale up from 0.5 L laboratory scale to 2000 L production scale for mesenchymal stem cell production. Based on your analysis:

a) What changes would you make to agitation and aeration parameters?
b) How would you expect cell density to change?
c) What would be the economic impact per million cells?

*Write your strategy here:*


---

## **Question 3: Parameter Optimization**
Using the optimization analysis, explain why different objectives (productivity, cost, density) require different operating conditions. Which trade-offs are most significant?

*Write your analysis here:*


---

## **Question 4: Bioreactor Selection**
Compare stirred tank vs hollow fiber bioreactors for these applications:

a) **Large-scale chondrocyte production** (for cartilage repair)
b) **Small-scale hepatocyte culture** (for drug testing)
c) **Research-scale cardiomyocyte expansion** (for heart patches)

Justify your choices based on the comparison data.

*Write your recommendations here:*


---

## **Question 5: Economic Analysis**
Modify the scale-up parameters to design a business case for producing 10¹² cells per year. Consider:

a) What bioreactor configuration would you choose?
b) How many batches per year would you need?
c) What would be the annual production cost?
d) At what price per million cells would this be profitable?

*Write your business analysis here:*


---

## **Question 6: Connection to Chapter 2**
From Chapter 2.1.2 on bioreactors for scaffold-free constructs, explain how bioreactor design affects:

a) **Spheroid formation** (from Exercise 1)
b) **ECM production** (from Exercise 2)
c) **Overall tissue quality** for scaffold-free approaches

*Write your connections here:*



# Section 8: Extension Challenges (Optional)

For advanced students who want to explore further.

In [ ]:
# Extension Challenge: Multi-stage bioprocess design

def multi_stage_process(seed_volume, expansion_stages, final_volume):
    """
    Design a multi-stage scale-up process.
    
    Parameters:
    seed_volume: starting volume (L)
    expansion_stages: number of intermediate stages
    final_volume: target production volume (L)
    
    Returns:
    process design with costs and timelines
    """
    
    # Calculate stage volumes (geometric progression)
    volume_ratio = (final_volume / seed_volume) ** (1 / expansion_stages)
    
    stages = []
    current_volume = seed_volume
    total_time = 0
    total_cost = 0
    
    for stage in range(expansion_stages + 1):
        # Optimize conditions for this volume
        kLa = oxygen_transfer_coefficient(current_volume, 80, 0.2)  # Moderate conditions
        max_density = calculate_maximum_cell_density(kLa)
        costs = bioreactor_cost_model(current_volume, 'stirred_tank')
        
        # Culture time (scales with volume due to mass transfer)
        culture_time = 7 + np.log10(current_volume) * 2  # days
        
        # Cell yield
        cell_yield = max_density * current_volume * 1000 * 0.8  # 80% efficiency
        
        stage_data = {
            'stage': stage + 1,
            'volume_L': current_volume,
            'culture_time_days': culture_time,
            'max_density': max_density,
            'cell_yield': cell_yield,
            'capital_cost': costs['capital_cost_EUR'],
            'operating_cost': costs['total_daily_cost_EUR'] * culture_time
        }
        
        stages.append(stage_data)
        total_time += culture_time
        total_cost += stage_data['operating_cost']
        
        # Next stage volume
        if stage < expansion_stages:
            current_volume *= volume_ratio
    
    return {
        'stages': stages,
        'total_time_days': total_time,
        'total_operating_cost': total_cost,
        'final_yield': stages[-1]['cell_yield']
    }

# Design example process: 0.5 L to 1000 L in 4 stages
process = multi_stage_process(0.5, 4, 1000)

print("MULTI-STAGE PROCESS DESIGN")
print("=" * 50)
print(f"Scale-up: 0.5 L → 1000 L in {len(process['stages'])} stages")
print(f"Total process time: {process['total_time_days']:.1f} days")
print(f"Total operating cost: €{process['total_operating_cost']:,.0f}")
print(f"Final cell yield: {process['final_yield']:.1e} cells")
print(f"Cost per million cells: €{process['total_operating_cost'] / (process['final_yield'] / 1e6):.2f}")

print("\nStage Details:")
for stage in process['stages']:
    print(f"Stage {stage['stage']}: {stage['volume_L']:6.1f} L, "
          f"{stage['culture_time_days']:4.1f} days, "
          f"{stage['cell_yield']:.1e} cells, "
          f"€{stage['operating_cost']:,.0f}")

# Visualize process
plt.figure(figsize=(14, 8))

# Subplot 1: Volume progression
plt.subplot(2, 2, 1)
volumes = [s['volume_L'] for s in process['stages']]
stages_num = [s['stage'] for s in process['stages']]
plt.semilogy(stages_num, volumes, 'bo-', linewidth=3, markersize=8)
plt.xlabel('Stage Number', fontweight='bold')
plt.ylabel('Volume (L)', fontweight='bold')
plt.title('Volume Scale-up Progression', fontweight='bold')
plt.grid(True, alpha=0.3)

# Subplot 2: Cell yield progression
plt.subplot(2, 2, 2)
yields = [s['cell_yield'] for s in process['stages']]
plt.semilogy(stages_num, yields, 'go-', linewidth=3, markersize=8)
plt.xlabel('Stage Number', fontweight='bold')
plt.ylabel('Cell Yield', fontweight='bold')
plt.title('Cell Production Scale-up', fontweight='bold')
plt.grid(True, alpha=0.3)

# Subplot 3: Culture time per stage
plt.subplot(2, 2, 3)
times = [s['culture_time_days'] for s in process['stages']]
plt.bar(stages_num, times, alpha=0.7, color='orange')
plt.xlabel('Stage Number', fontweight='bold')
plt.ylabel('Culture Time (days)', fontweight='bold')
plt.title('Culture Time per Stage', fontweight='bold')
plt.grid(True, alpha=0.3)

# Subplot 4: Cumulative cost
plt.subplot(2, 2, 4)
cumulative_cost = np.cumsum([s['operating_cost'] for s in process['stages']])
plt.plot(stages_num, cumulative_cost, 'ro-', linewidth=3, markersize=8)
plt.xlabel('Stage Number', fontweight='bold')
plt.ylabel('Cumulative Cost (EUR)', fontweight='bold')
plt.title('Cost Accumulation', fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nExtension Ideas:")
print("1. Compare single-stage vs multi-stage economics")
print("2. Optimize number of stages for minimum cost")
print("3. Add contamination risk analysis")
print("4. Include harvest and purification steps")
print("5. Model different cell types with varying growth rates")

# Section 9: Summary and Key Takeaways

## **What You've Learned:**

1. **Scale-up Principles**: How mass transfer limitations affect large-scale bioprocesses
2. **Economic Analysis**: Trade-offs between capital costs, operating costs, and productivity
3. **Optimization Methods**: How to balance competing objectives in bioprocess design
4. **Technology Comparison**: Strengths and weaknesses of different bioreactor types
5. **Process Engineering**: Multi-stage design for efficient scale-up

## **Connections to Chapter 2:**

- **Section 2.1.2**: Bioreactors for scaffold-free constructs and their design principles
- **Figure 2.3**: Examples of stirred and hollow fiber bioreactor systems
- **Integration**: How bioreactor design affects spheroid formation and ECM production
- **Applications**: Supporting industrial-scale scaffold-free biofabrication

## **Next Steps:**

Continue with **Exercise 4: Bioink Property Comparison** to explore how different bioink types perform in 3D bioprinting applications.

---

## **Learning Assessment**

**Before finishing, make sure you can:**

- [ ] Explain why oxygen transfer becomes limiting at large scale
- [ ] Calculate scaling relationships for volume, mixing, and mass transfer
- [ ] Compare different bioreactor types for specific applications
- [ ] Optimize operating conditions for different objectives
- [ ] Analyze economic trade-offs in bioprocess scale-up
- [ ] Design multi-stage processes for efficient cell expansion

**Outstanding work completing Exercise 3!**